In [68]:

import nltk
from nltk.stem.porter import PorterStemmer 
from nltk.corpus import stopwords
import re
import pandas as pd
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('puntk')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading puntk: Package 'puntk' not found in index


False

Importing Libraries and dowloading the nltk packages

In [ ]:
data = pd.read_csv("SMSSpamDectection.csv",names=["label", "message"])

In [ ]:
data.shape

(5575, 2)

In [ ]:
data.describe

<bound method NDFrame.describe of         label                                            message
0     Column1                                            Column2
1         ham  Go until jurong point, crazy.. Available only ...
2         ham                      Ok lar... Joking wif u oni...
3        spam  Free entry in 2 a wkly comp to win FA Cup fina...
4         ham  U dun say so early hor... U c already then say...
...       ...                                                ...
5570     spam  This is the 2nd time we have tried 2 contact u...
5571      ham              Will Ã¼ b going to esplanade fr home?
5572      ham  Pity, * was in mood for that. So...any other s...
5573      ham  The guy did some bitching but I acted like i'd...
5574      ham                         Rofl. Its true to its name

[5575 rows x 2 columns]>

in Column1 we have the labels to classify ham and spam and column2 has text to understand wheather the message is spam or ham

In [ ]:
data.dtypes

label      object
message    object
dtype: object

In [69]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Apply stemming and preprocessing steps to the data

In [70]:
data

,label,message
0,Column1,Column2
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...
...,...,...
5570,spam,This is the 2nd time we have tried 2 contact u...
5571,ham,Will Ã¼ b going to esplanade fr home?
5572,ham,"Pity, * was in mood for that. So...any other s..."
5573,ham,The guy did some bitching but I acted like i'd...


In [73]:
#Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(data['message']).toarray()

Counter vectorizer is used to calculate the occurance of each word in the given corpus. 

In [74]:
X.shape

(5575, 5000)

The hyperparameter max_Features is decided based on the number of columns in X, here the X[1] = 8753. we can select a max features of 8753. However, we select the max = 5000, as the most important features to decide if the message is spam or not will be within the 5000 features

In [75]:
y = pd.get_dummies(data['label'])
y = y.iloc[:,1].values

to separate the labels from the data, we creat a dummy variable and then store the values in y

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_Test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_Test)

Applying Naive Bayes model to the data. 

In [77]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [78]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)

In [79]:
confusion_m

array([[137,   2],
       [ 10, 966]])

In [80]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)

In [81]:
accuracy

0.989237668161435

The accuracy of the model is 98.9\% for max_features = 5000